<a href="https://colab.research.google.com/github/fannijako/ML_course_hws/blob/main/ML_hazi_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance

     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 6.4 MB 27.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive

import yfinance as yahooFinance

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import keras
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input, Conv2D, MaxPooling2D
from tensorflow.keras.utils import plot_model, to_categorical, img_to_array
from tensorflow.keras.metrics import MeanSquaredError

from PIL import Image

In [4]:
df = yahooFinance.Ticker("TSLA")
df = df.history(period="max", auto_adjust=True)
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-06-29,3.800000,5.000000,3.508000,4.778000,93831500,0,0.0
2010-06-30,5.158000,6.084000,4.660000,4.766000,85935500,0,0.0
2010-07-01,5.000000,5.184000,4.054000,4.392000,41094000,0,0.0
2010-07-02,4.600000,4.620000,3.742000,3.840000,25699000,0,0.0
2010-07-06,4.000000,4.000000,3.166000,3.222000,34334500,0,0.0
...,...,...,...,...,...,...,...
2022-04-04,1089.380005,1149.910034,1072.530029,1145.449951,27345300,0,0.0
2022-04-05,1136.300049,1152.869995,1087.300049,1091.260010,26691700,0,0.0
2022-04-06,1073.469971,1079.000000,1027.699951,1045.760010,29782800,0,0.0


In [5]:
df = df['Close']
df = pd.DataFrame(df).to_numpy()

In [6]:
def split_sequence(sequence, n_steps):
	x, y = list(), list()
	
	for i in range(len(sequence)):
		end_ix = i + n_steps # Az adott szekvencia-darab utolsó eleme
		
		if end_ix > len(sequence)-1: # Ellenőrizzük nem értünk-e a teljes lista végére
			break
		
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix] # Kimentjük a szekvencia darabot és az utolsó elemet mint eredményváltozó
		x.append(seq_x)
		y.append(seq_y)
	
	return np.array(x), np.array(y)

In [7]:
x, y = split_sequence(df, 10)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True)

In [8]:
#Alexnet

input = Input(shape=(10,1))

x=Conv1D(filters=128, kernel_size=3, strides=1, activation='relu')(input)
x=MaxPooling1D(pool_size=2, strides=2)(x)
x=Conv1D(filters=128, kernel_size=3, strides=1, activation='relu')(x)
x =MaxPooling1D(pool_size=2, strides=2)(x)
x=Flatten()(x)

x=Dense(4096, activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(4096, activation='relu')(x)
x=Dropout(0.2)(x)

output = Dense(1, activation='linear')(x)

In [9]:
model = keras.Model(input, output)
model.compile(optimizer='adam', loss='MSE')
early_stopping_callback = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(x_train, y_train, epochs=200, validation_split=0.2, callbacks=[early_stopping_callback])

Epoch 1/200
63/63 [==============================] - 15s 36ms/step - loss: 2926.3152 - val_loss: 1016.3906
Epoch 2/200
63/63 [==============================] - 1s 23ms/step - loss: 809.1357 - val_loss: 1121.3232
Epoch 3/200
63/63 [==============================] - 2s 27ms/step - loss: 687.5440 - val_loss: 392.0752
Epoch 4/200
63/63 [==============================] - 1s 20ms/step - loss: 752.0078 - val_loss: 718.2732
Epoch 5/200
63/63 [==============================] - 1s 15ms/step - loss: 584.3673 - val_loss: 323.7473
Epoch 6/200
63/63 [==============================] - 1s 14ms/step - loss: 1336.8754 - val_loss: 509.6552
Epoch 7/200
63/63 [==============================] - 1s 15ms/step - loss: 853.0012 - val_loss: 1045.5051
Epoch 8/200
63/63 [==============================] - 1s 14ms/step - loss: 593.0462 - val_loss: 274.4294
Epoch 9/200
63/63 [==============================] - 1s 14ms/step - loss: 850.9961 - val_loss: 706.6307
Epoch 10/200
63/63 [==============================] - 1s 1

In [10]:
AlexNet=keras.Model(input, output)
model.evaluate(x_test, y_test)

14/14 [==============================] - 0s 6ms/step - loss: 178.1095


178.10951232910156

## Képek

In [11]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
class_names = ['repulo', 'auto', 'madar', 'macska', 'szarvas', 'kutya', 'beka', 'lo', 'hajo', 'kamion']

170508288/170498071 [==============================] - 6s 0us/step


In [12]:
drive.mount('/content/drive')
lo = plt.imread('/content/drive/My Drive/ML kurzus/ló.jpg')
bicikli = plt.imread('/content/drive/My Drive/ML kurzus/bicikli.jpg')
helikopter = plt.imread('/content/drive/My Drive/ML kurzus/helikopter.jpg')

Mounted at /content/drive


In [13]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = to_categorical(y_train, num_classes=len(class_names))
y_test = to_categorical(y_test, num_classes=len(class_names))

In [14]:
input = Input(shape=(32,32,3))

x = Conv2D(filters=64, kernel_size=5, strides=4, activation='relu', padding='same')(input)
x = MaxPooling2D(pool_size=2, strides=2)(x)
x = Conv2D(filters=256, kernel_size=5, activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=2, strides=2)(x)
x = Conv2D(filters=384, kernel_size=3, activation='relu', padding='same')(x)
x = Conv2D(filters=384, kernel_size=3, activation='relu', padding='same')(x)
x = Conv2D(filters=192, kernel_size=3, activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=2, strides=2)(x)
x = Flatten()(x)

x = Dense(4096, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.4)(x)

output = Dense(len(class_names), activation='softmax')(x)

In [15]:
AlexNet = keras.Model(input, output)
early_stopping_callback = EarlyStopping(patience=3, restore_best_weights=True)
tensorboard_callback = TensorBoard('logs/run1')
AlexNet.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
AlexNet.fit(x_train, y_train, epochs=100, validation_split=0.1,callbacks=[early_stopping_callback, tensorboard_callback])

Epoch 1/100
1407/1407 [==============================] - 38s 26ms/step - loss: 2.3031 - accuracy: 0.1020 - val_loss: 2.3029 - val_accuracy: 0.0976
Epoch 2/100
1407/1407 [==============================] - 35s 25ms/step - loss: 2.3028 - accuracy: 0.1005 - val_loss: 2.3027 - val_accuracy: 0.0970
Epoch 3/100
1407/1407 [==============================] - 35s 25ms/step - loss: 2.3028 - accuracy: 0.0970 - val_loss: 2.3031 - val_accuracy: 0.0950
Epoch 4/100
1407/1407 [==============================] - 35s 25ms/step - loss: 2.3028 - accuracy: 0.0991 - val_loss: 2.3028 - val_accuracy: 0.0976
Epoch 5/100
1407/1407 [==============================] - 35s 25ms/step - loss: 2.3028 - accuracy: 0.0982 - val_loss: 2.3029 - val_accuracy: 0.0986


In [16]:
AlexNet.predict(x_test)

array([[0.09830204, 0.09844925, 0.10091174, ..., 0.10322863, 0.09797035,
        0.10038666],
       [0.09830204, 0.09844925, 0.10091174, ..., 0.10322863, 0.09797035,
        0.10038666],
       [0.09830204, 0.09844925, 0.10091174, ..., 0.10322863, 0.09797035,
        0.10038666],
       ...,
       [0.09830204, 0.09844925, 0.10091174, ..., 0.10322863, 0.09797035,
        0.10038666],
       [0.09830204, 0.09844925, 0.10091174, ..., 0.10322863, 0.09797035,
        0.10038666],
       [0.09830204, 0.09844925, 0.10091174, ..., 0.10322863, 0.09797035,
        0.10038666]], dtype=float32)

In [17]:
AlexNet.evaluate(x_test, y_test)

313/313 [==============================] - 3s 8ms/step - loss: 2.3027 - accuracy: 0.1000


[2.302722930908203, 0.10000000149011612]

In [19]:
lo = Image.open('/content/drive/My Drive/ML kurzus/ló.jpg')
helikopter = Image.open('/content/drive/My Drive/ML kurzus/helikopter.jpg')
bicikli = Image.open('/content/drive/My Drive/ML kurzus/bicikli.jpg')

In [20]:
lo = lo.resize((32, 32))
lo = img_to_array(lo)

helikopter = helikopter.resize((32, 32))
helikopter = img_to_array(helikopter)

bicikli = bicikli.resize((32, 32))
bicikli = img_to_array(bicikli)

pics = np.array([lo, helikopter, bicikli])
pics = pics.astype('float32') / 255

In [23]:
AlexNet.predict(pics)

array([[0.09830204, 0.09844925, 0.10091174, 0.09993123, 0.09858728,
        0.10220612, 0.10002668, 0.10322863, 0.09797035, 0.10038666],
       [0.09830204, 0.09844925, 0.10091174, 0.09993123, 0.09858728,
        0.10220612, 0.10002668, 0.10322863, 0.09797035, 0.10038666],
       [0.09830204, 0.09844925, 0.10091174, 0.09993123, 0.09858728,
        0.10220612, 0.10002668, 0.10322863, 0.09797035, 0.10038666]],
      dtype=float32)